In [144]:
import tkinter as tk
from tkinter import ttk
import sqlite3
sql_connect = sqlite3.connect('database.db')
cur = sql_connect.cursor() # creates a cursor object 

cur.execute("DROP TABLE IF EXISTS app_data")

cur.execute('''
CREATE TABLE IF NOT EXISTS app_data (
    taskId INTEGER,
    task TEXT,
    category TEXT
)
''')
sql_connect.commit()


#create main window
main = tk.Tk()

#create title for app
main.title("Task Manager System")

#sizing for app
main.geometry("500x400")

label = tk.Label(main, text="Welcome to the Task Manager System!", font=('Arial', 16))
label.pack()

add_task = tk.StringVar()
task_Id = tk.IntVar()

#displaying this messgae if user tries to enter a task with no text in the box
def error_msg():
    top = tk.Toplevel(main)
    top_label = tk.Label(top, text='Error. Please enter a task.')
    top_label.pack()

#submitting task function
def submit():
    task = add_task.get()
    category = combo.get()
    taskId = task_Id.get()
#returns error popup 
    if task == "":
        error_msg()
        return
    
    query = "INSERT INTO app_data (taskId, task, category) VALUES (?, ?, ?)"
    cur.execute(query, (taskId, task, category))
    sql_connect.commit()

    display_text = f"{taskId}: {task} ({category})"
    task_list.insert(tk.END, display_text)

    add_task.set("")
    task_Id.set(0)

def delete_task():
    selection = task_list.curselection()
    if not selection:
        print("No task selected to delete")
        return
    index = selection[0]
    item_text = task_list.get(index)

    # item_text looks like: "5: Take out trash (Chore)"
    task_id_str = item_text.split(":", 1)[0]

    try:
        task_id_int = int(task_id_str)
    except ValueError:
        print("Could not parse task ID")
        return    

    cur.execute("DELETE FROM app_data WHERE taskId = ?", (task_id_int,))
    sql_connect.commit()

    # Delete from listbox
    task_list.delete(index)


#loading tasks from the DB
def load_tasks():
    cur.execute("SELECT task, taskId, category FROM app_data")
    rows = cur.fetchall()
    for row in rows:
        display_text = f"{row[1]}: {row[0]} ({row[2]})"
        task_list.insert(tk.END, display_text)


# label above task
task_label = tk.Label(main,text="Enter your task:", font=('Arial', 12))
task_label.pack(side=tk.TOP)
# user input for task name
task_desc = tk.Entry(main, textvariable=add_task, width=10, font=('Arial', 12))
task_desc.pack(side=tk.TOP)

task_importance = tk.Label(main, text="Enter the level of importance:", font=('Arial', 12))
task_importance.pack(side=tk.TOP)

task_id = tk.Entry(main, textvariable=task_Id, width=10, font=('Arial', 12))
task_id.pack(side=tk.TOP)

type_label = tk.Label(main, text="Choose your task type:", font=('Arial', 12))
type_label.pack(side=tk.TOP)

# Create a Combobox widget
combo = ttk.Combobox(main)

# task category
combo['values'] = ('Work', 'Chore', 'School')

# Set a default value for the Combobox
combo.set("Work")

# Pack the Combobox widget
combo.pack(side=tk.TOP)

#submit button for adding tasks
submit_button = tk.Button(main, text = "Add Task",
                           activebackground="lightblue",
                             activeforeground="darkblue",
                             cursor="hand2",
                             command=submit)
submit_button.pack(side=tk.TOP)

#list box that holds the tasks
task_list = tk.Listbox(main, height=5, width=40, font=('Arial', 12))
task_list.pack(side=tk.TOP)

#creating delete button for task
delete_button = tk.Button(main, text = "Delete Task",
                           activebackground="lightblue",
                            activeforeground="darkblue",
                             cursor="hand2",
                             command=delete_task)
delete_button.pack()






load_tasks()

main.mainloop()


No task selected to delete


In [145]:
import pandas as pd
pd.read_sql_query('''
    SELECT * FROM app_data;
    ''', sql_connect)

,taskId,task,category
0,100,Do homework,Chore
